In [1]:
import xarray as xr
import numpy as np
import os

Load the data in the `xarray` fashion:

In [2]:
root      = "/home/nate/data/ILAMB/MODELS/CLM/CLM5.0/"
nbp_file  = "nbp_Lmon_CLM50r267GSWP3_historical_r1i1p1_185001-201012.nc"
area_file = "areacella_fx_CLM45r270GSWP3_historical_r0i0p0.nc"
dset_nbp  = xr.open_dataset(os.path.join(root, nbp_file))
dset_area = xr.open_dataset(os.path.join(root,area_file))
nbp       = dset_nbp["nbp"]
area      = dset_area.areacella

Also create some masked arrays, equivalent to what is happening behind `integrateInSpace()` from ILAMB.

In [3]:
N = np.ma.masked_invalid(nbp.data)
A = area.data

Now we define a few methods of computed the area-weighted sum, which is an rectangle rule approximation of the integral.

In [4]:
def method1():
    s = (nbp*area).sum(area.dims) 
    return s

def method2():
    weighted_nbp = nbp.weighted(area)
    s = weighted_nbp.sum(area.dims) 
    return s

def method3():
    s = (N*A).sum(axis=(-2,-1))
    return s

s1 = method1()
s2 = method2()
s3 = method3()
print("relative difference = %.3e" % (np.abs(((s2-s1)/s1).data).max()))
print("relative difference = %.3e" % (np.abs(((s3-s1)/s1).data).max()))

relative difference = 2.559e-05
relative difference = 0.000e+00


In [5]:
%timeit method1()

297 ms ± 5.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit method2()

234 ms ± 5.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit method3()

400 ms ± 4.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
xr.__version__

'0.15.1'